In [1]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sionna.phy


import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.phy.config.seed = 42 # Set seed for reproducible results

E0000 00:00:1745207948.760485   49016 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745207948.764852   49016 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745207948.777741   49016 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745207948.777753   49016 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745207948.777755   49016 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745207948.777757   49016 computation_placer.cc:177] computation placer already registered. Please check linka

In [2]:
import sys
sys.path.append('../')
from src import SystemConfig, UeConfig, MyConfig, MySimulator, MyPUSCHConfig

In [3]:
from src.my_pusch_config import MyPUSCHConfig
from src.my_encoder import MyTBEncoder
from src.my_decoder import MyTBDecoder

from sionna.phy.mapping import BinarySource, Mapper
from sionna.phy.nr import PUSCHPilotPattern, LayerMapper, LayerDemapper, PUSCHLSChannelEstimator
from sionna.phy.ofdm import ResourceGrid, ResourceGridMapper, LinearDetector
from sionna.phy.nr.utils import generate_prng_seq
from sionna.phy.channel import AWGN, OFDMChannel, gen_single_sector_topology as gen_topology
from sionna.phy.mimo import StreamManagement
from sionna.phy.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, CDL
from sionna.phy import Block


import tensorflow as tf
import numpy as np

In [13]:
sys_cfg = SystemConfig(
    NCellId=442
)
ue_cfg = UeConfig()
my_cfg = MyConfig(sys_cfg, ue_cfg)
my_pusch_cfg = MyPUSCHConfig(my_config=my_cfg)

In [14]:
my_pusch_cfg.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 442
n_size_grid : 162
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 1944, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (1944, 14)
dmrs_symbol_indices : [3, 11]
first_resource_block : 0
first_subcarrier : 0
frequency_hopping : neither
l : [3, 11]
l_0 : 3
l_bar : [3, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (486,)
n_rnti : 20002
n_size_bwp : 162
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 46656
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 162
num_subcarriers : 1944
phy_cell_id : 442
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [

In [15]:
my_simulator = MySimulator(my_pusch_cfg)
my_simulator

In [16]:
my_simulator.tbEnc.scrambler.c_init

[655425978]

In [17]:
tf.config.run_functions_eagerly(True)

In [18]:
b,c,y = my_simulator(1,1e-10)

In [ ]:
my_simulator

In [22]:
class MyGenerator(Block):
    def __init__(self,
                 my_simulator:MySimulator=None,
                 precision=None,
                 **kwargs):
        super().__init__(precision=precision, **kwargs)
        self.sim = my_simulator

    def call(self):
        b,c,y = self.sim(3, 1e-3)
        return b,c,y
my_generator = MyGenerator(my_simulator)
my_generator()

(<tf.Tensor: shape=(3, 1, 5512), dtype=float32, numpy=
 array([[[1., 0., 1., ..., 1., 1., 0.]],
 
        [[1., 1., 1., ..., 1., 1., 0.]],
 
        [[1., 0., 1., ..., 1., 0., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(3, 1, 46656), dtype=float32, numpy=
 array([[[1., 0., 1., ..., 1., 1., 0.]],
 
        [[1., 1., 0., ..., 0., 1., 1.]],
 
        [[0., 0., 0., ..., 0., 1., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(3, 1, 4, 14, 1944), dtype=complex64, numpy=
 array([[[[[ 9.72134888e-01-3.20031911e-01j,
             3.67315680e-01+9.32381094e-01j,
            -3.84636670e-01-9.14988220e-01j, ...,
            -1.27208352e+00-1.87673166e-01j,
             1.26213312e+00+2.00772703e-01j,
             1.30951655e+00+1.91579804e-01j],
           [ 3.36748689e-01+9.25950527e-01j,
             9.53760922e-01-3.49360406e-01j,
            -3.35273087e-01-8.80055964e-01j, ...,
            -2.12726206e-01+1.27281594e+00j,
            -1.75315008e-01+1.26080704e+00j,
             1.79833934e-01-1